# Analyze Product Sentiment

In [2]:
import turicreate

# Read product review data

In [38]:
products = turicreate.SFrame('./amazon_baby.sframe')

# Building a sentiment classifier

## Build word count vectors

In [80]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [41]:
products

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'handles': 1.0,'stripping': 1.0, ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'disappointed': 1.0, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'the':1.0, 'than': 1.0, 'fu ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'binky': 2.0, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ..."


# Define what is positive and negative sentiment

In [81]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [82]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [44]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'disappointed': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'the':1.0, 'than': 1.0, 'fu ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'binky': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",1


# Train our sentiment classifier

In [83]:
train_data,test_data = products.random_split(.8,seed=0)

In [84]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 2.673560     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 3.074801     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 3.665733     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 5.693627     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [85]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.9176975738650012,
 'auc': 0.9342357833151299,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      1       |        0        |  1344 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.33047871872210066,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.

### Accuracy of sentiment model 'accuracy': 0.9176975738650012

In [86]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [87]:
def word_count (word_dict, word):
    if not (word in word_dict):
        return 0
    return word_dict[word]

In [88]:
for word in selected_words :
    products[word] = products['word_count'].apply(lambda x: word_count(x,word))

# Question 1

In [89]:
for word in selected_words :
    print(word, products[word].sum())

awesome 3892.0
great 55791.0
fantastic 1664.0
amazing 2628.0
love 41994.0
horrible 1110
bad 4183
terrible 1146.0
awful 687
wow 425
hate 1107


In [90]:
selected_words_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features= selected_words, validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.545102     | 0.847708          | 0.846385            |

| 5         | 6        | 0.658063     | 0.847708          | 0.846385            |

| 6         | 7        | 0.779738     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

# Question 2


In [127]:
sentiment_model.coefficients.sort('index').print_rows(num_rows=1000, num_columns=4)
##sentiment_model.coefficients.print_rows(num_rows=1000, num_columns=4)

+-------------+----------------------+-------+-----------------------+-----+
|     name    |        index         | class |         value         | ... |
+-------------+----------------------+-------+-----------------------+-----+
| (intercept) |         None         |   1   |   1.240508137457956   | ... |
|  word_count |          0           |   1   |  -0.7701126717681165  | ... |
|  word_count |          00          |   1   |  -0.9437426024788287  | ... |
|  word_count |         000          |   1   |  -0.7176877760475436  | ... |
|  word_count |         0001         |   1   |   2.4808091275882385  | ... |
|  word_count |        000ft         |   1   |   2.228440789477914   | ... |
|  word_count |         001          |   1   |   -7.906440589853358  | ... |
|  word_count |        001cm         |   1   |   0.3334804853930616  | ... |
|  word_count |         00am         |   1   |   0.6832922895372402  | ... |
|  word_count |      00amcreepy      |   1   |   -5.822364509703614  | ... |

# Question 3

In [118]:
selected_words_model.coefficients.sort('value' , ascending= False)

name,index,class,value,stderr
love,None,1,1.3592688669224757,0.028068300152099997
(intercept),None,1,1.3365913848878148,0.008929969787657532
awesome,None,1,1.1335346660341103,0.08399643983187567
amazing,None,1,1.1000933113659914,0.09954776260465965
fantastic,None,1,0.8858047568813963,0.11167591293399713
great,None,1,0.8630655001195999,0.018955052444380473
wow,None,1,-0.00953823606771224,0.1604641122471162
bad,None,1,-0.9914778800650894,0.03848428664699065
hate,None,1,-1.3484407222463402,0.07715698604297318
awful,None,1,-2.05290820403138,0.1009973543525922


In [92]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8463848186404036,
 'auc': 0.6936022046674926,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  159  |
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.3962265467087835,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 532

### Answer is 0.841 to 0.871

# Calculate accuracy of major classifier

In [94]:
total_positive = 159 + 27817

In [98]:
accuracy = total_positive/test_data.num_rows()

In [99]:
accuracy

0.8400192169108815

In [100]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [101]:
diaper_champ_reviews

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'convenient': 1.0,'more': 1.0, 'trash': ...",1
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'system': 1.0, 'try':1.0, 're': 1.0, 'still': ...",0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'nose': 1.0, 'for': 2.0,'investment': 1.0, ...",1
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'out': 1.0, 'pull': 1.0,'open': 1.0, 'pail': ...",1
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'winter': 1.0,'outside': 1.0, 'day': ...",1
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'mom': 1.0, 'huge': 1.0,'special': 1.0, 'good': ...",1
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'yuck': 1.0, 'clean':1.0, 'trash': 3.0, 'is': ...",0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1.0,'suggestions': 1.0, ...",1
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'best': 1.0, 'that':1.0, 'will': 1.0, ...",1
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'no': 1.0, 'regular':1.0, 'part': 1.0, ...",1


In [102]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type = 'probability')

In [103]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [104]:
diaper_champ_reviews[0]

{'name': 'Baby Trend Diaper Champ',
 'review': "I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out.",
 'rating': 4.0,
 'word_count': {'key': 1.0,
  'have': 1.0,
  'pieces': 1.0,
  'between': 1.0,
  'cushioned': 1.0,
  'piece': 1.0,
  'take': 1.0,
  'overloading': 1.0,
  'be': 1.0,
  'looking': 1.0,
  're': 1.0,
  'stinkiness': 1.0,
  'retain': 1.0,
  'now': 1.0,
  'wonderful': 1.0,
  'worked': 1.0,
  '16': 1.0,
  'and': 3.0,
  'months': 1.0,
  've': 1.0,
  'in': 1.0,
  'us': 1.0,
  'i': 3.0,
  'experienced': 1.0,
  '

In [106]:
diaper_champ_reviews['predicted_sentiment2'] = selected_words_model.predict(diaper_champ_reviews, output_type = 'probability')

In [111]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [112]:
diaper_champ_reviews[0]

{'name': 'Baby Trend Diaper Champ',
 'review': "I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out.",
 'rating': 4.0,
 'word_count': {'key': 1.0,
  'have': 1.0,
  'pieces': 1.0,
  'between': 1.0,
  'cushioned': 1.0,
  'piece': 1.0,
  'take': 1.0,
  'overloading': 1.0,
  'be': 1.0,
  'looking': 1.0,
  're': 1.0,
  'stinkiness': 1.0,
  'retain': 1.0,
  'now': 1.0,
  'wonderful': 1.0,
  'worked': 1.0,
  '16': 1.0,
  'and': 3.0,
  'months': 1.0,
  've': 1.0,
  'in': 1.0,
  'us': 1.0,
  'i': 3.0,
  'experienced': 1.0,
  '